### Gradio

In [2]:
!pip install gradio


   ---------------------------------------- 0.0/23.0 MB ? eta -:--:--
   ---------- ----------------------------- 6.0/23.0 MB 28.4 MB/s eta 0:00:01
   ----------------- ---------------------- 10.2/23.0 MB 24.5 MB/s eta 0:00:01
   --------------------- ------------------ 12.1/23.0 MB 19.4 MB/s eta 0:00:01
   ------------------------ --------------- 14.2/23.0 MB 16.8 MB/s eta 0:00:01
   ---------------------------- ----------- 16.3/23.0 MB 15.5 MB/s eta 0:00:01
   ------------------------------- -------- 18.4/23.0 MB 14.3 MB/s eta 0:00:01
   ----------------------------------- ---- 20.4/23.0 MB 13.8 MB/s eta 0:00:01
   ---------------------------------------  22.5/23.0 MB 13.2 MB/s eta 0:00:01
   ---------------------------------------- 23.0/23.0 MB 12.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 16.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
   --

In [4]:
# gradio_app.py
import gradio as gr
import matplotlib.pyplot as plt


def run_pipeline(tiff_file, voxel_z, voxel_y, voxel_x, particle_dia, frame_interval):
    image = load_tiff_stack(tiff_file.name)
    detections_df = basic_detect(image, (voxel_z, voxel_y, voxel_x), particle_dia)

    if image.shape[0] > 1:
        tracked_df = track_with_trackpy(detections_df, voxel_size_um=(voxel_z, voxel_y, voxel_x))
        msd_df = compute_msd(tracked_df, frame_interval=frame_interval)

        fig, ax = plt.subplots()
        ax.plot(msd_df['lag_time'], msd_df['msd'], marker='o')
        ax.set_xlabel("Lag time (s)")
        ax.set_ylabel("MSD (µm²)")
        ax.set_title("Mean Squared Displacement")
        return fig
    else:
        return "Single 3D stack detected, tracking not performed."

iface = gr.Interface(
    fn=run_pipeline,
    inputs=[
        gr.File(label="Upload TIFF"),
        gr.Number(label="Voxel Z (µm)", value=1.0),
        gr.Number(label="Voxel Y (µm)", value=1.0),
        gr.Number(label="Voxel X (µm)", value=1.0),
        gr.Number(label="Particle diameter (µm)", value=5.0),
        gr.Number(label="Frame interval (s)", value=1.0)
    ],
    outputs=gr.Plot()
)

iface.launch()


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
